# 2.1 - GOES16 - Download

In [1]:
import autoroot
import os
from dotenv import load_dotenv
from pathlib import Path
import earthaccess
import numpy as np
import xarray as xr
from dataclasses import dataclass
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

# PLOTTING
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import rasterio
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns
sns.reset_defaults()
sns.set_context(context="talk", font_scale=0.7)



xr.set_options(
    keep_attrs=True, 
    display_expand_data=False, 
    display_expand_coords=False, 
    display_expand_data_vars=False, 
    display_expand_indexes=False
)
np.set_printoptions(threshold=10, edgeitems=2)

%load_ext autoreload
%autoreload 2

## ROI

The first thing we need to do is find a region of interest.
We are going to do a very simple example where we define a bounding box.

In [2]:
from odc.geo.xr import xr_coords
from odc.geo.geom import BoundingBox
from odc.geo.geobox import GeoBox
from odc.geo.geom import Geometry
from odc.geo.crs import CRS

from odc.geo.data import ocean_geom, country_geom

We will do a simple bounding box around our area of interest.

### Download GOES16

GOES16 is a geostationary satellite so there are very frequent observations.
The data is typically stored as full fields so we need to be smart when selecting the datasets.
For this example, we will only download the data that is available from the MODIS satellite.

In [3]:
from rs_tools._src.data.modis import MODISFileName, MODIS_ID_TO_NAME, MODIS_NAME_TO_ID

/home/juanjohn/miniconda/envs/rs_tools/lib/python3.11/site-packages/goes2go/data.py:519: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  within=pd.to_timedelta(config["nearesttime"].get("within", "1H")),
/home/juanjohn/miniconda/envs/rs_tools/lib/python3.11/site-packages/goes2go/NEW.py:188: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  within=pd.to_timedelta(config["nearesttime"].get("within", "1H")),


In [5]:
# environment variables
save_dir = Path(os.getenv("ITI_DATA_SAVEDIR"))
modis_read_dir = Path("/pool/usuarios/juanjohn/data/iti/pipeline_test/modis/clean")
goes_save_dir = Path("/pool/usuarios/juanjohn/data/iti/pipeline_test/goes16/raw")
modis_read_dir

PosixPath('/pool/usuarios/juanjohn/data/iti/pipeline_test/modis/clean')

We will subset each of them to extract the timestamps when they were retrieved.

In [6]:
from rs_tools._src.utils.io import get_list_filenames

In [7]:
# list of all .hdf files in the directory
all_modis_files = get_list_filenames(modis_read_dir, ".nc")

# filter files for terra files ONLY ( our satellite id)
modis_files = list(filter(lambda x: "aqua" in x, all_modis_files))

len(modis_files)

12

In [8]:
time_stamp = Path(modis_files[0]).name.split("_")[0]
time_stamp = datetime.strptime(time_stamp, "%Y%m%d%H%M%S")
print(f"Time Stamp: {time_stamp}")

Time Stamp: 2020-10-01 15:00:00


We have some extra tools to enable us to select the appropriate satellite.
This is simply syntatic sugar because these names can be a bit difficult to remember.

In [9]:
def find_nearest_time(target_time, time_list):
    # Calculate the difference between each time in the list and the target time
    time_differences = [abs(time - target_time) for time in time_list]
    
    # Find the index of the smallest difference
    nearest_time_index = time_differences.index(min(time_differences))
    
    # Return the nearest time
    return time_list[nearest_time_index]

In [10]:
# GOES2GO
import goes2go
from goes2go import GOES
from rs_tools._src.data.goes.download import goes_download
import pandas as pd

In [11]:
# GOES has 16 bands
bands = list(np.arange(1, 17))

In [12]:
# ABI Level 1b Data
G16_DL = GOES(
    satellite=16, 
    product="ABI-L1b-Rad", 
    domain='F',
    channel=bands
)


In [13]:
# ABI Level 1b Data
G16_CM = GOES(
    satellite=16, 
    product="ABI-L2-ACM", 
    domain='F',
    channel=bands
)

In [14]:
attime = str(time_stamp)
attime

'2020-10-01 15:00:00'

In [15]:
within = pd.to_timedelta(10, unit="minutes")
within

Timedelta('0 days 00:10:00')

#### Inspection

GOES allows us to check which files are available for download before we need to download anyfiles.
There might not be any need but it might be helpful just to see what is available.

In [18]:
%%time
# download image
within = pd.to_timedelta(10, unit="minute")
goes_images_files = G16_DL.nearesttime(
    attime=attime,
    within=within,
    return_as="filelist",
    save_dir=goes_save_dir,
    download=False

)
# goes_imag = goes_imag.assign_coords({"t": goes_imag["band_wavelength"].squeeze()})
# goes_imag = goes_imag.drop_vars(["band_wavelength"])
# goes_imag = goes_imag.rename({"t": "band_wavelength"})

# # correction
# p = goes_imag["goes_imager_projection"]

# # The projection x and y coordinates equals the scanning angle (in radians)
# # multiplied by the satellite height See details here:
# # https://proj4.org/operations/projections/geos.html?highlight=geostationary
# goes_imag["x"] = goes_imag.FOV.x
# goes_imag["y"] = goes_imag.FOV.y
# goes_imag = goes_imag.rio.write_crs(goes_imag.FOV.crs, inplace=True)
goes_images_files

/home/juanjohn/miniconda/envs/rs_tools/lib/python3.11/site-packages/goes2go/data.py:136: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  DATES = pd.date_range(f"{start:%Y-%m-%d %H:00}", f"{end:%Y-%m-%d %H:00}", freq="1H")


CPU times: user 32.3 ms, sys: 1.68 ms, total: 34 ms
Wall time: 525 ms


,file,product_mode,satellite,start,end,creation,product,mode_bands,mode,band
0,noaa-goes16/ABI-L1b-RadF/2020/275/15/OR_ABI-L1...,ABI-L1b-RadF-M6C14,G16,2020-10-01 15:00:19.600,2020-10-01 15:09:50.400,2020-10-01 15:09:59.900,ABI-L1b-RadF,M6C14,6,14
1,noaa-goes16/ABI-L1b-RadF/2020/275/15/OR_ABI-L1...,ABI-L1b-RadF-M6C13,G16,2020-10-01 15:00:19.600,2020-10-01 15:09:51.500,2020-10-01 15:09:59.000,ABI-L1b-RadF,M6C13,6,13
2,noaa-goes16/ABI-L1b-RadF/2020/275/15/OR_ABI-L1...,ABI-L1b-RadF-M6C12,G16,2020-10-01 15:00:19.600,2020-10-01 15:09:50.900,2020-10-01 15:09:56.900,ABI-L1b-RadF,M6C12,6,12
3,noaa-goes16/ABI-L1b-RadF/2020/275/15/OR_ABI-L1...,ABI-L1b-RadF-M6C11,G16,2020-10-01 15:00:19.600,2020-10-01 15:09:50.400,2020-10-01 15:09:57.400,ABI-L1b-RadF,M6C11,6,11
4,noaa-goes16/ABI-L1b-RadF/2020/275/15/OR_ABI-L1...,ABI-L1b-RadF-M6C10,G16,2020-10-01 15:00:19.600,2020-10-01 15:09:51.500,2020-10-01 15:09:58.300,ABI-L1b-RadF,M6C10,6,10
5,noaa-goes16/ABI-L1b-RadF/2020/275/15/OR_ABI-L1...,ABI-L1b-RadF-M6C09,G16,2020-10-01 15:00:19.600,2020-10-01 15:09:50.900,2020-10-01 15:09:57.900,ABI-L1b-RadF,M6C09,6,9
6,noaa-goes16/ABI-L1b-RadF/2020/275/15/OR_ABI-L1...,ABI-L1b-RadF-M6C08,G16,2020-10-01 15:00:19.600,2020-10-01 15:09:50.400,2020-10-01 15:09:57.800,ABI-L1b-RadF,M6C08,6,8
7,noaa-goes16/ABI-L1b-RadF/2020/275/15/OR_ABI-L1...,ABI-L1b-RadF-M6C05,G16,2020-10-01 15:00:19.600,2020-10-01 15:09:50.400,2020-10-01 15:09:56.200,ABI-L1b-RadF,M6C05,6,5
8,noaa-goes16/ABI-L1b-RadF/2020/275/15/OR_ABI-L1...,ABI-L1b-RadF-M6C06,G16,2020-10-01 15:00:19.600,2020-10-01 15:09:51.000,2020-10-01 15:09:53.600,ABI-L1b-RadF,M6C06,6,6
9,noaa-goes16/ABI-L1b-RadF/2020/275/15/OR_ABI-L1...,ABI-L1b-RadF-M6C04,G16,2020-10-01 15:00:19.600,2020-10-01 15:09:50.400,2020-10-01 15:09:55.600,ABI-L1b-RadF,M6C04,6,4


### All TimeStamps

Now, we can do the same thing by for all MODIS time stamps available.

In [19]:
from tqdm.auto import tqdm

within = pd.to_timedelta(10, unit="minute")

In [24]:
pbar = tqdm(modis_files)
goes_image_files = list()
for ifile in pbar:

    time_stamp = Path(ifile).name.split("_")[0]
    time_stamp = datetime.strptime(time_stamp, "%Y%m%d%H%M%S")

    goes_image_files.append(G16_DL.nearesttime(
        attime=time_stamp,
        within=within,
        return_as="filelist",
        save_dir=goes_save_dir,
        download=False
    ))

    

  0%|          | 0/12 [00:00<?, ?it/s]

/home/juanjohn/miniconda/envs/rs_tools/lib/python3.11/site-packages/goes2go/data.py:136: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  DATES = pd.date_range(f"{start:%Y-%m-%d %H:00}", f"{end:%Y-%m-%d %H:00}", freq="1H")
/home/juanjohn/miniconda/envs/rs_tools/lib/python3.11/site-packages/goes2go/data.py:136: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  DATES = pd.date_range(f"{start:%Y-%m-%d %H:00}", f"{end:%Y-%m-%d %H:00}", freq="1H")
/home/juanjohn/miniconda/envs/rs_tools/lib/python3.11/site-packages/goes2go/data.py:136: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  DATES = pd.date_range(f"{start:%Y-%m-%d %H:00}", f"{end:%Y-%m-%d %H:00}", freq="1H")
/home/juanjohn/miniconda/envs/rs_tools/lib/python3.11/site-packages/goes2go/data.py:136: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' 

In [33]:
goes_image_files_ = pd.concat(goes_image_files, axis=0)

In [36]:
len(goes_image_files_)

192

#### Downloading

In [17]:
%%time
# download image
within = pd.to_timedelta(10, unit="minute")
goes_images_files = G16_DL.nearesttime(
    attime=attime,
    within=within,
    return_as="filelist",
    save_dir=goes_save_dir,
    download=False

)
# goes_imag = goes_imag.assign_coords({"t": goes_imag["band_wavelength"].squeeze()})
# goes_imag = goes_imag.drop_vars(["band_wavelength"])
# goes_imag = goes_imag.rename({"t": "band_wavelength"})

# # correction
# p = goes_imag["goes_imager_projection"]

# # The projection x and y coordinates equals the scanning angle (in radians)
# # multiplied by the satellite height See details here:
# # https://proj4.org/operations/projections/geos.html?highlight=geostationary
# goes_imag["x"] = goes_imag.FOV.x
# goes_imag["y"] = goes_imag.FOV.y
# goes_imag = goes_imag.rio.write_crs(goes_imag.FOV.crs, inplace=True)
goes_images_files

/home/juanjohn/miniconda/envs/rs_tools/lib/python3.11/site-packages/goes2go/data.py:136: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  DATES = pd.date_range(f"{start:%Y-%m-%d %H:00}", f"{end:%Y-%m-%d %H:00}", freq="1H")


📦 Finished downloading [16] files to [/pool/usuarios/juanjohn/data/iti/pipeline_test/goes16/raw/noaa-goes16/ABI-L1b-RadF].
CPU times: user 1.92 s, sys: 1.53 s, total: 3.44 s
Wall time: 45.2 s


,file,product_mode,satellite,start,end,creation,product,mode_bands,mode,band
0,noaa-goes16/ABI-L1b-RadF/2020/275/15/OR_ABI-L1...,ABI-L1b-RadF-M6C14,G16,2020-10-01 15:00:19.600,2020-10-01 15:09:50.400,2020-10-01 15:09:59.900,ABI-L1b-RadF,M6C14,6,14
1,noaa-goes16/ABI-L1b-RadF/2020/275/15/OR_ABI-L1...,ABI-L1b-RadF-M6C13,G16,2020-10-01 15:00:19.600,2020-10-01 15:09:51.500,2020-10-01 15:09:59.000,ABI-L1b-RadF,M6C13,6,13
2,noaa-goes16/ABI-L1b-RadF/2020/275/15/OR_ABI-L1...,ABI-L1b-RadF-M6C12,G16,2020-10-01 15:00:19.600,2020-10-01 15:09:50.900,2020-10-01 15:09:56.900,ABI-L1b-RadF,M6C12,6,12
3,noaa-goes16/ABI-L1b-RadF/2020/275/15/OR_ABI-L1...,ABI-L1b-RadF-M6C11,G16,2020-10-01 15:00:19.600,2020-10-01 15:09:50.400,2020-10-01 15:09:57.400,ABI-L1b-RadF,M6C11,6,11
4,noaa-goes16/ABI-L1b-RadF/2020/275/15/OR_ABI-L1...,ABI-L1b-RadF-M6C10,G16,2020-10-01 15:00:19.600,2020-10-01 15:09:51.500,2020-10-01 15:09:58.300,ABI-L1b-RadF,M6C10,6,10
5,noaa-goes16/ABI-L1b-RadF/2020/275/15/OR_ABI-L1...,ABI-L1b-RadF-M6C09,G16,2020-10-01 15:00:19.600,2020-10-01 15:09:50.900,2020-10-01 15:09:57.900,ABI-L1b-RadF,M6C09,6,9
6,noaa-goes16/ABI-L1b-RadF/2020/275/15/OR_ABI-L1...,ABI-L1b-RadF-M6C08,G16,2020-10-01 15:00:19.600,2020-10-01 15:09:50.400,2020-10-01 15:09:57.800,ABI-L1b-RadF,M6C08,6,8
7,noaa-goes16/ABI-L1b-RadF/2020/275/15/OR_ABI-L1...,ABI-L1b-RadF-M6C05,G16,2020-10-01 15:00:19.600,2020-10-01 15:09:50.400,2020-10-01 15:09:56.200,ABI-L1b-RadF,M6C05,6,5
8,noaa-goes16/ABI-L1b-RadF/2020/275/15/OR_ABI-L1...,ABI-L1b-RadF-M6C06,G16,2020-10-01 15:00:19.600,2020-10-01 15:09:51.000,2020-10-01 15:09:53.600,ABI-L1b-RadF,M6C06,6,6
9,noaa-goes16/ABI-L1b-RadF/2020/275/15/OR_ABI-L1...,ABI-L1b-RadF-M6C04,G16,2020-10-01 15:00:19.600,2020-10-01 15:09:50.400,2020-10-01 15:09:55.600,ABI-L1b-RadF,M6C04,6,4


In [37]:
pbar = tqdm(modis_files)
goes_image_files = list()
for ifile in pbar:

    time_stamp = Path(ifile).name.split("_")[0]
    time_stamp = datetime.strptime(time_stamp, "%Y%m%d%H%M%S")

    goes_image_files.append(G16_DL.nearesttime(
        attime=time_stamp,
        within=within,
        return_as="filelist",
        save_dir=goes_save_dir,
        download=True
    ))


  0%|          | 0/12 [00:00<?, ?it/s]

/home/juanjohn/miniconda/envs/rs_tools/lib/python3.11/site-packages/goes2go/data.py:136: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  DATES = pd.date_range(f"{start:%Y-%m-%d %H:00}", f"{end:%Y-%m-%d %H:00}", freq="1H")


 👮🏻‍♂️ File already exists. Do not overwrite: /pool/usuarios/juanjohn/data/iti/pipeline_test/goes16/raw/noaa-goes16/ABI-L1b-RadF/2020/275/15/OR_ABI-L1b-RadF-M6C14_G16_s20202751500196_e20202751509504_c20202751509599.nc
 👮🏻‍♂️ File already exists. Do not overwrite: /pool/usuarios/juanjohn/data/iti/pipeline_test/goes16/raw/noaa-goes16/ABI-L1b-RadF/2020/275/15/OR_ABI-L1b-RadF-M6C13_G16_s20202751500196_e20202751509515_c20202751509590.nc
 👮🏻‍♂️ File already exists. Do not overwrite: /pool/usuarios/juanjohn/data/iti/pipeline_test/goes16/raw/noaa-goes16/ABI-L1b-RadF/2020/275/15/OR_ABI-L1b-RadF-M6C12_G16_s20202751500196_e20202751509509_c20202751509569.nc
 👮🏻‍♂️ File already exists. Do not overwrite: /pool/usuarios/juanjohn/data/iti/pipeline_test/goes16/raw/noaa-goes16/ABI-L1b-RadF/2020/275/15/OR_ABI-L1b-RadF-M6C11_G16_s20202751500196_e20202751509504_c20202751509574.nc
 👮🏻‍♂️ File already exists. Do not overwrite: /pool/usuarios/juanjohn/data/iti/pipeline_test/goes16/raw/noaa-goes16/ABI-L1b-RadF

/home/juanjohn/miniconda/envs/rs_tools/lib/python3.11/site-packages/goes2go/data.py:136: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  DATES = pd.date_range(f"{start:%Y-%m-%d %H:00}", f"{end:%Y-%m-%d %H:00}", freq="1H")


 👮🏻‍♂️ File already exists. Do not overwrite: /pool/usuarios/juanjohn/data/iti/pipeline_test/goes16/raw/noaa-goes16/ABI-L1b-RadF/2020/275/15/OR_ABI-L1b-RadF-M6C01_G16_s20202751500196_e20202751509504_c20202751509565.nc 👮🏻‍♂️ File already exists. Do not overwrite: /pool/usuarios/juanjohn/data/iti/pipeline_test/goes16/raw/noaa-goes16/ABI-L1b-RadF/2020/275/15/OR_ABI-L1b-RadF-M6C02_G16_s20202751500196_e20202751509504_c20202751509548.nc

 👮🏻‍♂️ File already exists. Do not overwrite: /pool/usuarios/juanjohn/data/iti/pipeline_test/goes16/raw/noaa-goes16/ABI-L1b-RadF/2020/275/15/OR_ABI-L1b-RadF-M6C03_G16_s20202751500196_e20202751509504_c20202751509569.nc
 👮🏻‍♂️ File already exists. Do not overwrite: /pool/usuarios/juanjohn/data/iti/pipeline_test/goes16/raw/noaa-goes16/ABI-L1b-RadF/2020/275/15/OR_ABI-L1b-RadF-M6C05_G16_s20202751500196_e20202751509504_c20202751509562.nc
 👮🏻‍♂️ File already exists. Do not overwrite: /pool/usuarios/juanjohn/data/iti/pipeline_test/goes16/raw/noaa-goes16/ABI-L1b-RadF

...

📦 Finished downloading [16] files to [/pool/usuarios/juanjohn/data/iti/pipeline_test/goes16/raw/noaa-goes16/ABI-L1b-RadF].


/home/juanjohn/miniconda/envs/rs_tools/lib/python3.11/site-packages/goes2go/data.py:136: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  DATES = pd.date_range(f"{start:%Y-%m-%d %H:00}", f"{end:%Y-%m-%d %H:00}", freq="1H")


📦 Finished downloading [16] files to [/pool/usuarios/juanjohn/data/iti/pipeline_test/goes16/raw/noaa-goes16/ABI-L1b-RadF].


/home/juanjohn/miniconda/envs/rs_tools/lib/python3.11/site-packages/goes2go/data.py:136: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  DATES = pd.date_range(f"{start:%Y-%m-%d %H:00}", f"{end:%Y-%m-%d %H:00}", freq="1H")


📦 Finished downloading [16] files to [/pool/usuarios/juanjohn/data/iti/pipeline_test/goes16/raw/noaa-goes16/ABI-L1b-RadF].


Now, we can download the data using the EarthAccess framework.
The main thing we need is the period and (optional) a bounding box.
We don't want to download too much data so we will give a bounding box of interest.